In [391]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline

Data = pd.read_csv('train.csv')
Test = pd.read_csv('test.csv')
gender = pd.read_csv('gender_submission.csv')
def split_func(x):
    if 'LINE' in x:
        return 0
    elif ' ' in x:
        return int(x.split(' ')[-1])
    else:
        return int(x)
    
def embarked(x):
    if x == 'S':
        return 1
    elif x == 'C':
        return 2
    else:
        return 3
    
def normalize(x, feature, data):
    return (x - data[feature].mean())/(data[feature].max() - data[feature].min())
    
Data.Ticket = Data.Ticket.map(lambda x: split_func(x))
Test.Ticket = Test.Ticket.map(lambda x: split_func(x))
Data.Ticket = Data.Ticket.map(lambda x: normalize(x, 'Ticket', data = Data))
Test.Ticket = Test.Ticket.map(lambda x: normalize(x, 'Ticket', data = Test))
Data.Fare = Data.Fare.map(lambda x: normalize(x, 'Fare', data = Data))
Test.Fare = Test.Fare.map(lambda x: normalize(x, 'Fare', data = Test))
Data.Sex = (Data.Sex == 'male')
Test.Sex = (Test.Sex == 'male')
Data.Sex = Data.Sex.map(lambda x: 1 if x == True else 0)
Test.Sex = Test.Sex.map(lambda x: 1 if x == True else 0)
Data.Embarked = Data.Embarked.map(lambda x: embarked(x))
Test.Embarked = Test.Embarked.map(lambda x: embarked(x))
Data.loc[np.isnan(Data.Age), 'Age'] = Data.Age.mean()
Test.loc[np.isnan(Test.Age), 'Age'] = Test.Age.mean()
Data.Age = Data.Age.map(lambda x: normalize(x, 'Age', data = Data))
Test.Age = Test.Age.map(lambda x: normalize(x, 'Age', data = Test))

X_train_orig = Data.drop(['Cabin', 'Name', 'Survived', 'PassengerId'], axis = 1)
Test_orig = Test.drop(['Cabin', 'Name', 'PassengerId'], axis = 1)
y_train = Data.Survived

In [362]:
X_train = np.hstack((np.ones(shape = (X_train_orig.shape[0], 1)), X_train_orig.values))
X_test = np.hstack((np.ones(shape = (Test_orig.shape[0], 1)), Test_orig.values))
param = np.random.normal(size = (X_train.shape[1]))

def logistic(w, x):
    return (1+np.exp(-np.dot(w.T, x)))**-1

def obj(w, X, y):
    obj = 0
    for dp in range(y.shape[0]):
        x_dp = X[dp, :]
        y_dp = logistic(w, x_dp)
        t_dp = y[dp]
        obj += -t_dp*np.log(y_dp) + (t_dp - 1)*np.log(1 - y_dp)
    return obj

def compute_y(w, X, R):
    y_dp_arr = np.zeros((X.shape[0]))
    for dp in range(X.shape[0]):
        x_dp = X[dp, :]
        y_dp = logistic(w, x_dp)
        y_dp_arr[dp] = y_dp
    return y_dp_arr

def compute_grad(w, X, R, y, y_arr):
    grad = np.dot(X.T, y_arr - y)
    return grad

def update_weights(w, X, y, R):
    y_arr = compute_y(w, X, R)
    grad = compute_grad(w, X, R, y, y_arr)
    return grad

def optimise(w, X, y, R):
    for i in range(5000):
        w = w - (0.001)*update_weights(w, X, y, R)   
    return w
    
w = optimise(param, X_train, y_train, R)     

In [404]:
output = np.zeros((X_test.shape[0]))

for i in range(X_test.shape[0]):
    k = logistic(w, X_test[i, :])
    if k >= 0.5:
        output[i] = int(1)
    else:
        output[i] = int(0)

output
output_pd = pd.DataFrame(output, index = Test.PassengerId, columns = ['Survived'])
output_pd.astype(int).to_csv('my_output_3.csv')
np.sum((output == gender.Survived))/gender.shape[0]
output_pd = output_pd.astype(int)
output_pd.to_csv('Titanic.csv')

(891, 9)